
try to make this clever

llm scaffold?


In [ ]:

const PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';

async function llmScaffold(github) {

  if(!github) {
    console.error('Provide a github repository for comparison and updating')
  }



  // TODO: feed the llm the proper project components as necessary to get it to generate code files
  

}

module.exports = llmScaffold




llm code?


In [ ]:




import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##helpful coding module\nnamed Llama that responds to every request using beautifully crafted `markdown`. Return only a single code block in the specified language without reasoning or instructions if requested.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'DeepSeek-R1-Distill-Llama-70B-Q5_K_S.gguf' ),
          //contextSize: 2048
      });
  }

  await initSession()

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

async function llmCode(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default llmCode




bash project files?


In [ ]:

const fs = require('fs')
const path = require('path')
const PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
const {spawnSync} = require('child_process')
const llama = importer.import('llm code')
const {safeurl} = importer.import('domain cache tools')

async function generateBash(prompt, github) {
  const llmCode = (await llama)

  // TODO: local pull
  if(!fs.existsSync(github)
    && fs.existsSync(path.join(PROFILE_PATH, github))) {
      github = path.join(PROFILE_PATH, github)
  }

  if(github.includes('://')
    && fs.existsSync(path.basename(github).replace('.git', ''))
  ) {
    github = path.join(PROFILE_PATH, path.basename(github).replace('.git', ''))
  }

  if(!fs.existsSync(github)) {
    return
  }

  let baseName = path.basename(github)

  await spawnSync('git', ['pull'], {
    cwd: github,
    timeout: 3000,
    stdio: ['pipe', 'pipe', 'pipe']
  })

  // TODO: generate bash
  let q1 = 'Project listing:\n' +  fs.readdirSync(github).join('\n')
    + '\nGenerate a Bash shell script for the following request, include any empty file touch/folder creations, file moves or general file management: ' + prompt + '\n</think>Don\'t do any file content insertion or reasoning, limit response to two or three shell command lines.'
  console.log('User: ' + q1)
  let a1 = await llmCode(q1)
  console.log('AI: ' + a1)

  // try to extract code blocks
  let code = a1.matchAll(/```(bash|javascript|code)*\n[\s\S]*?\n```/gi)

  // make sure we're oporating on our own directory
  let codeBlocks = '#!/bin/bash\ncd /home/' + baseName + '\n\n'

  // extract code blocks from response
  for(let match of code) {
    codeBlocks += match[0].replace(/^```(bash|javascript|code)*\n|\n```$/gi, '') + '\n'
  }
  if(!codeBlocks) {
    console.log('Error, couldn\'t find code in:' + a1)
    return
  }

  if(!fs.existsSync(path.join(github, '.automations'))) {
    fs.mkdirSync(path.join(github, '.automations'))
  }
  const CODE_FILE = path.join('.automations', safeurl((new Date).toISOString() + '-' + prompt) + '.sh')
  fs.writeFileSync(path.join(github, CODE_FILE), codeBlocks)
  fs.chmodSync(path.join(github, CODE_FILE), 0o777)

  // TODO: execute new docker file on project
  let args = ['run', '-it', '--rm', '-v', 
    // TODO: mount project directory
    github + ':/home/' + baseName, 
    // TODO: change docker image
    'alpine:latest', 'sh', 
    path.join('/home/', baseName, CODE_FILE)]
  console.log('docker', args.join(' '))
  await spawnSync('docker', args, {
    cwd: github,
    timeout: 3000,
    stdio: [0, 1, 2]
  })

  // TODO: git commit and git push?

}

module.exports = generateBash



generate code?


In [ ]:


async function generateCode(github) {
  
  // TODO: local pull

  // TODO: compare existing project files

  // TODO: create new files

  // TODO: populate code files with goals

  // TODO: populate goals with actual code

}

module.exports = generateCode

